#Step0: Import

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import layers
from gensim.models import Word2Vec
import pickle

In [ ]:
base_path = '/content/gdrive/MyDrive/Colab Notebooks/Bioinformatics Course/'

In [ ]:
KMER_SIZE = 6

#Step1: Getting the data

In [ ]:
drive.mount('/content/gdrive')
training_set = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Bioinformatics Course/training_set.csv')
development_set = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Bioinformatics Course/development_set.csv')
test_set = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Bioinformatics Course/test_set.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<div dir="rtl" >
  <font color="yellow" size=4.5>
  تعداد داده هر دیتاست 
  </font>
</div>

In [ ]:
print(training_set.shape)
print(development_set.shape)
print(test_set.shape)

(1320, 2)
(180, 2)
(400, 1)


#Step2: Preprocessing 

<div dir="rtl" >
  <font color="yellow" size=4.5>
  تابع زیر رشته را به عنوان ورودی می‌گیرید (همراه با سایز k) و خروجی آرایه ای که شامل kmers ها هستند را برمی‌گرداند.
  </font>
</div>

In [ ]:
def build_kmers(sequence, ksize=KMER_SIZE):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)

    return kmers

<div dir="rtl" >
  <font color="yellow" size=4.5>
در قسمت زیر دیکشنری از kmer های هر رشته و لیبل آن رشته برای دیتاست training می‌سازیم
  </font>
</div>

In [ ]:
kmer_label_training_dict = {"kmer": [], "label": []}
for index, row in training_set.iterrows():
  seq = row['Sequence']
  label = row['Type']
  kmer = build_kmers(seq)
  kmer_label_training_dict['kmer'].append(kmer)
  kmer_label_training_dict['label'].append(label)

<div dir="rtl" >
  <font color="yellow" size=4.5>
در قسمت زیر دیکشنری از kmer های هر رشته و لیبل آن رشته برای دیتاست develop می‌سازیم
  </font>
</div>

In [ ]:
kmer_label_develop_dict = {"kmer": [], "label": []}
for index, row in development_set.iterrows():
  seq = row['Sequence']
  label = row['Type']
  kmer = build_kmers(seq)
  kmer_label_develop_dict['kmer'].append(kmer)
  kmer_label_develop_dict['label'].append(label)

<div dir="rtl" >
  <font color="yellow" size=4.5>
در قسمت زیر دیکشنری از kmer های هر رشته برای دیتاست test می‌سازیم
  </font>
</div>

In [ ]:
kmer_test_dict = {"kmer": []}
for index, row in test_set.iterrows():
  seq = row['Sequence']
  kmer = build_kmers(seq)
  kmer_test_dict['kmer'].append(kmer)

In [ ]:
X_train, Y_train = kmer_label_training_dict['kmer'], kmer_label_training_dict['label']
X_develop, Y_develop = kmer_label_develop_dict['kmer'],kmer_label_develop_dict['label']
X_test = kmer_test_dict['kmer']

<div dir="rtl" >
  <font color="yellow" size=4.5>
لیبل هایمان که همان شماره کلاس هستند، قسمت کلاس را از آنها حذف میکنیم و از 1 تا 6 به 0 تا 5 منتقل میکنیم
  </font>
</div>

In [ ]:
train_labels = []
for label in Y_train:
  train_labels.append(int(label[5]) - 1)

In [ ]:
develop_labels = []
for label in Y_develop:
  develop_labels.append(int(label[5]) - 1)

In [ ]:
print(len(train_labels))
print(len(develop_labels))

1320
180


<div dir="rtl" >
  <font color="yellow" size=4.5>
مدل word to vector
  </font>
</div>

In [ ]:
w2v_model = Word2Vec(X_train, min_count=1, window=5, size=100, workers=2)

<div dir="rtl" >
  <font color="yellow" size=4.5>
بردار مربوط به TACCAC را مشاهده می‌کنید
  </font>
</div>

In [ ]:
sample_seq = X_train[0]
sample_seq_kmer = sample_seq[0]
print(f'sample kmer: {sample_seq_kmer}')
print(f'Sample kmer w2v: {w2v_model.wv[sample_seq_kmer]}')

sample kmer: TACCAC
Sample kmer w2v: [ 5.1599485e-01 -1.3614917e+00 -2.5137069e+00  2.8759272e+00
 -2.9731245e+00  1.0125425e+00  1.8879658e+00 -2.6912949e+00
  2.6431346e-01  1.2701818e+00 -4.8164029e+00 -1.7164217e+00
 -1.0497183e+00 -9.0810478e-01 -4.0334481e-01  2.0055683e+00
  1.6849827e+00 -3.8675990e+00  4.5742302e+00 -1.6496680e+00
  2.0198862e-01  1.8451649e+00 -6.3169706e-01  1.4565941e+00
  1.0978358e+00 -2.6513109e+00  1.7381676e+00 -2.3525589e+00
  3.7699294e+00 -1.4200971e+00  2.5649369e+00 -2.5167935e+00
 -2.1253190e+00  2.7088518e+00 -3.0681753e+00  1.7813683e+00
 -2.5602813e+00 -2.1462939e+00  6.2966142e+00 -4.1152456e-01
 -3.6921275e+00  5.2292066e+00  1.3471276e+00 -1.5122943e+00
 -1.1167527e+00 -3.7736675e-01 -3.8644547e+00  9.6770442e-01
  8.4013861e-01  4.1194940e+00 -3.6208923e+00 -2.3320251e+00
 -1.6333948e-01  3.3926473e+00 -4.3795338e+00 -5.5680981e+00
 -1.4624577e+00  6.2226816e-03 -1.8758857e+00 -7.9645365e-01
 -1.5029290e+00  1.9100444e-01 -1.0418806e+00 -3

<div dir="rtl" >
  <font color="yellow" size=4.5>
حال می‌خواهیم هر رشته seq را به برداری به طول 100 که طول word to vector مان 
بود، تبدیل کنیم
در آخر هم  بردار را نرمال میکنیم.
در آخر هم آرایه را برمیگردانیم. (در ابتدا یک خانه اضافه کرده بودیم و در انتها آن خانه را در نظر نمیگیریم)
  </font>
</div>

In [ ]:
def create_seqs_embedding(w2v_model, seqs):
  seqs_embedding = np.array([np.zeros(100)])
  for seq in seqs: 
    vector = np.zeros(100)
    for word in seq: 
      vector += w2v_model.wv[word]
    number_of_word_in_seq = len(seq)
    normalized_vector = vector / number_of_word_in_seq
    seqs_embedding = np.append(seqs_embedding, [normalized_vector], axis=0)
  
  return seqs_embedding[1: ]

In [ ]:
X_train_embedding = create_seqs_embedding(w2v_model, X_train)

In [ ]:
X_develop_embedding = create_seqs_embedding(w2v_model, X_develop)

In [ ]:
X_test_embedding = create_seqs_embedding(w2v_model, X_test)

<div dir="rtl" >
  <font color="yellow" size=4.5>
ابعاد هر کدام را مشاهده می‌کنید
  </font>
</div>

In [ ]:
print(X_train_embedding.shape)
print(X_develop_embedding.shape)
print(X_test_embedding.shape)

(1320, 100)
(180, 100)
(400, 100)


#Step3: Build Model

<div dir="rtl" >
  <font color="yellow" size=4.5>
در زیر لایه های شبکه عصبی را مشاهده می‌کنید، برای جلوگیری از overfitting از dropout با ضریب 0.2 استفاده شده است.
  </font>
</div>

In [ ]:
model = keras.Sequential([
                             
                             keras.layers.Dense(units=100, activation='relu', input_shape=(100,)),
                             keras.layers.Dense(units=50, activation='relu'),
                             keras.layers.Dropout(rate=0.2),
                             keras.layers.Dense(units=20, activation='relu'),
                             keras.layers.Dense(units=6, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', 
              loss=tf.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
train_labels = keras.utils.to_categorical(train_labels)
train_labels.shape

(1320, 6)

In [ ]:
develop_labels = keras.utils.to_categorical(develop_labels)
develop_labels.shape

(180, 6)

In [ ]:
history = model.fit(
    X_train_embedding,
    train_labels,
    epochs=10
)

Epoch 1/10
42/42 [==============================] - 1s 3ms/step - loss: 1.4103 - accuracy: 0.5644
Epoch 2/10
42/42 [==============================] - 0s 3ms/step - loss: 0.5503 - accuracy: 0.9061
Epoch 3/10
42/42 [==============================] - 0s 3ms/step - loss: 0.1606 - accuracy: 0.9826
Epoch 4/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0624 - accuracy: 0.9985
Epoch 5/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0285 - accuracy: 0.9992
Epoch 6/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0188 - accuracy: 0.9992
Epoch 7/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0117 - accuracy: 1.0000
Epoch 8/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0104 - accuracy: 1.0000
Epoch 9/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 10/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0105 - accuracy: 0.9985


<div dir="rtl" >
  <font color="yellow" size=4.5>
  حال مدل آمورش دیده شده را روی development تست میکنیم تا مدل را ارزیابی کنیم.
  </font>
</div>

#Step4: check model on development set

<div dir="rtl" >
  <font color="yellow" size=4.5>
صحت را بر دیتاست development بررسی میکنیم که برابر 1 است
  </font>
</div>

In [ ]:
development_loss, development_acc = model.evaluate(X_develop_embedding,  develop_labels)
print('development accuracy:', development_acc)


6/6 [==============================] - 0s 3ms/step - loss: 0.0013 - accuracy: 1.0000
development accuracy: 1.0


#Step5: Prediction on test set

In [ ]:
model_predicted = model.predict(X_test_embedding)

In [ ]:
predict = []
for i in range(len(model_predicted)):
  predict.append(np.argmax(model_predicted[i]))


In [ ]:
print(predict)

[4, 4, 2, 0, 5, 0, 5, 2, 3, 0, 3, 2, 1, 3, 3, 2, 5, 2, 2, 2, 3, 0, 4, 5, 0, 4, 2, 2, 2, 2, 0, 2, 5, 0, 5, 0, 2, 4, 4, 5, 4, 3, 4, 4, 2, 5, 3, 4, 0, 1, 3, 2, 3, 2, 2, 5, 0, 5, 4, 2, 0, 2, 2, 2, 2, 5, 3, 4, 5, 5, 5, 2, 1, 5, 0, 0, 3, 5, 5, 1, 4, 3, 0, 0, 2, 4, 5, 3, 2, 4, 2, 0, 0, 5, 4, 2, 4, 0, 3, 0, 2, 2, 1, 0, 2, 3, 2, 2, 2, 2, 2, 0, 3, 0, 3, 3, 2, 2, 2, 3, 5, 0, 5, 3, 4, 2, 5, 1, 0, 2, 2, 5, 4, 1, 1, 2, 1, 0, 2, 2, 4, 0, 5, 0, 1, 1, 0, 2, 0, 0, 5, 5, 1, 5, 2, 4, 4, 2, 5, 2, 0, 1, 3, 3, 2, 2, 4, 3, 0, 0, 1, 2, 5, 0, 2, 1, 5, 3, 4, 5, 4, 5, 4, 4, 5, 2, 2, 2, 5, 3, 5, 2, 2, 5, 5, 5, 0, 0, 4, 0, 0, 2, 2, 0, 0, 0, 4, 2, 1, 1, 0, 0, 0, 0, 2, 5, 2, 2, 3, 4, 5, 3, 0, 3, 2, 3, 5, 4, 3, 5, 4, 4, 5, 5, 3, 4, 2, 5, 5, 2, 0, 0, 3, 3, 3, 4, 0, 1, 5, 0, 5, 2, 1, 1, 0, 1, 2, 5, 5, 4, 0, 2, 5, 0, 2, 2, 0, 0, 4, 3, 0, 2, 0, 1, 0, 0, 2, 2, 2, 5, 0, 5, 3, 0, 2, 3, 2, 2, 3, 2, 0, 2, 5, 5, 1, 0, 5, 1, 0, 5, 5, 0, 3, 0, 5, 4, 5, 2, 3, 1, 3, 2, 0, 5, 4, 4, 1, 2, 4, 3, 4, 0, 2, 2, 3, 0, 0, 0, 0, 1, 0, 4, 1, 